# Chapter 24: Parser Combinators

In [65]:
:!stack install trifecta parsers

In [2]:
import Text.Trifecta

In [3]:
stop :: Parser a
stop = unexpected "stop"

In [4]:
one :: Parser Char
one = char '1'

In [5]:
one' :: Parser a
one' = one >> stop

In [6]:
oneTwo = char '1' >> char '2'

oneTwo' = oneTwo >> stop

testParse' :: String -> Parser Char -> Result Char
testParse' s p = parseString p mempty s

testParse :: Parser Char -> Result Char
testParse = testParse' "123"

In [7]:
:t parseString

parseString :: forall a. Parser a -> Delta -> String -> Result a

In [8]:
testParse stop

Failure (ErrInfo {_errDoc = (interactive):1:1: error: unexpected
    stop
123<EOF> 
^        , _errDeltas = [Columns 0 0]})

In [9]:
testParse one

Success '1'

In [10]:
testParse one'

Failure (ErrInfo {_errDoc = (interactive):1:2: error: unexpected
    stop
123<EOF> 
 ^       , _errDeltas = [Columns 1 1]})

In [11]:
testParse oneTwo

Success '2'

In [12]:
testParse oneTwo'

Failure (ErrInfo {_errDoc = (interactive):1:3: error: unexpected
    stop
123<EOF> 
  ^      , _errDeltas = [Columns 2 2]})

In [13]:
:t eof

eof :: forall (m :: * -> *). Parsing m => m ()

In [14]:
one'' = one <* eof
testParse one''

Failure (ErrInfo {_errDoc = (interactive):1:2: error: expected: end of input
123<EOF> 
 ^       , _errDeltas = [Columns 1 1]})

In [15]:
testParse' "1" one''

Success '1'

In [16]:
oneTwo'' = oneTwo <* eof
testParse oneTwo''

Failure (ErrInfo {_errDoc = (interactive):1:3: error: expected: end of input
123<EOF> 
  ^      , _errDeltas = [Columns 2 2]})

In [17]:
testParse' "12" oneTwo''

Success '2'

In [18]:
:t string

string :: forall (m :: * -> *). CharParsing m => String -> m String

In [19]:
parseString (string "123") mempty "123"

Success "123"

In [20]:
parseString (string "1") mempty "123"

Success "1"

In [21]:
parseString (string "12") mempty "123"

Success "12"

In [22]:
parseString (string "123") mempty "123"

Success "123"

In [23]:
string' :: String -> Parser String
string' = traverse char

In [24]:
parseString (string' "123") mempty "123"

Success "123"

In [25]:
import Control.Applicative

In [26]:
:info Alternative

class Applicative f => Alternative (f :: * -> *) where
  empty :: f a
  (<|>) :: f a -> f a -> f a
  some :: f a -> f [a]
  many :: f a -> f [a]
  {-# MINIMAL empty, (<|>) #-}
  	-- Defined in ‘GHC.Base’
instance Alternative m => Alternative (Unspaced m) -- Defined in ‘Text.Parser.Token’
instance Alternative m => Alternative (Unlined m) -- Defined in ‘Text.Parser.Token’
instance Alternative m => Alternative (Unhighlighted m) -- Defined in ‘Text.Parser.Token’
instance Alternative ZipList -- Defined in ‘Control.Applicative’
instance GHC.Base.MonadPlus m => Alternative (WrappedMonad m) -- Defined in ‘Control.Applicative’
instance (Control.Arrow.ArrowZero a, Control.Arrow.ArrowPlus a) => Alternative (WrappedArrow a b) -- Defined in ‘Control.Applicative’
instance Alternative Result -- Defined in ‘Text.Trifecta.Result’
instance Alternative Parser -- Defined in ‘Text.Trifecta.Parser’
instance Alternative [] -- Defined in ‘GHC.Base’
instance Alternative Maybe -- Defined in ‘GHC.Base’
instance Alternative IO -- Defined in ‘GHC.Base’
instance [safe] Control.Monad.Trans.Error.Error e => Alternative (Either e) -- Defined in ‘Control.Monad.Trans.Error’

In [27]:
:info Parser

newtype Parser a
  = Parser {unparser :: forall r.
                        (a -> Err -> Text.Trifecta.Util.It.It Rope r)
                        -> (Err -> Text.Trifecta.Util.It.It Rope r)
                        -> (a -> Data.Set.Internal.Set String -> Text.Trifecta.Delta.Delta -> Data.ByteString.Internal.ByteString -> Text.Trifecta.Util.It.It Rope r)
                        -> (ErrInfo -> Text.Trifecta.Util.It.It Rope r)
                        -> Text.Trifecta.Delta.Delta
                        -> Data.ByteString.Internal.ByteString
                        -> Text.Trifecta.Util.It.It Rope r}
  	-- Defined in ‘Text.Trifecta.Parser’
instance Functor Parser -- Defined in ‘Text.Trifecta.Parser’
instance (Semigroup a, Monoid a) => Monoid (Parser a) -- Defined in ‘Text.Trifecta.Parser’
instance Semigroup a => Semigroup (Parser a) -- Defined in ‘Text.Trifecta.Parser’
instance Alternative Parser -- Defined in ‘Text.Trifecta.Parser’
instance Applicative Parser -- Defined in ‘Text.Trifecta.Parser’
instance Monad Parser -- Defined in ‘Text.Trifecta.Parser’
instance CharParsing Parser -- Defined in ‘Text.Trifecta.Parser’
instance DeltaParsing Parser -- Defined in ‘Text.Trifecta.Parser’
instance Errable Parser -- Defined in ‘Text.Trifecta.Parser’
instance Parsing Parser -- Defined in ‘Text.Trifecta.Parser’
instance TokenParsing Parser -- Defined in ‘Text.Trifecta.Parser’

In [28]:
parseString (stop <|> one) mempty "123"

Success '1'

In [29]:
parseString (one <|> stop) mempty "123"

Success '1'

In [30]:
parser = (string "123" <* eof) <|> (string "12" <* eof) <|> (string "1" <* eof)

In [31]:
parseString parser mempty "123"

Success "123"

In [32]:
parseString parser mempty "12"

Success "12"

In [33]:
parseString parser mempty "1"

Success "1"

In [34]:
parseString parser mempty "1234"

Failure (ErrInfo {_errDoc = (interactive):1:4: error: expected: end of input
1234<EOF> 
   ^      , _errDeltas = [Columns 3 3]})

In [35]:
parseString parser mempty "2"

Failure (ErrInfo {_errDoc = (interactive):1:1: error: expected: "1",
    "12", "123"
2<EOF> 
^      , _errDeltas = [Columns 0 0]})

---

In [36]:
badFraction = "1/0"
alsoBad = "10"
shouldWork = "1/2"
shouldAlsoWork = "2/1"

In [38]:
import Data.Ratio ((%))

parseFraction :: Parser Rational
parseFraction = do
    numerator <- decimal
    char '/'
    denominator <- decimal
    return (numerator % denominator)

In [40]:
parseString parseFraction mempty shouldWork
parseString parseFraction mempty shouldAlsoWork

Success (1 % 2)

Success (2 % 1)

In [41]:
parseString parseFraction mempty badFraction

: 

In [53]:
parseString parseFraction mempty alsoBad

Failure (ErrInfo {_errDoc = (interactive):1:3: error: unexpected
    EOF, expected: "/", digit
10<EOF> 
  ^     , _errDeltas = [Columns 2 2]})

In [62]:
data EnsureResult
    = EnsureSuccess
    | EnsureFailure String

ensureM' :: Monad m => (a -> EnsureResult) -> a -> m a
ensureM' p a = case p a of
    EnsureSuccess -> return a
    EnsureFailure reason -> fail reason

mkEnsureP :: String -> (a -> Bool) -> (a -> EnsureResult)
mkEnsureP reason p = f . p where
    f False = EnsureFailure reason
    f True = EnsureSuccess
    
ensureM :: Monad m => String -> (a -> Bool) -> a -> m a
ensureM reason p = ensureM' (mkEnsureP reason p)

ensureNotZero :: Monad m => Integer -> m Integer
ensureNotZero = ensureM "must not be zero" (/=0)

decimalNonZero :: Parser Integer
decimalNonZero = decimal >>= ensureNotZero        

In [63]:
virtuousFraction :: Parser Rational
virtuousFraction = do
    numerator <- decimal
    char '/'
    denominator <- decimalNonZero
    return (numerator % denominator)
    
parseString virtuousFraction mempty badFraction
parseString virtuousFraction mempty shouldWork

Failure (ErrInfo {_errDoc = (interactive):1:4: error: must
    not be zero, expected: digit
1/0<EOF> 
   ^     , _errDeltas = [Columns 3 3]})

Success (1 % 2)

In [66]:
parseString (integer <* eof) mempty "123"
parseString (integer <* eof) mempty "123abc"

Success 123

Failure (ErrInfo {_errDoc = (interactive):1:4: error: expected: digit,
    end of input
123abc<EOF> 
   ^        , _errDeltas = [Columns 3 3]})

In [68]:
:t skipMany

skipMany :: forall (m :: * -> *) a. Parsing m => m a -> m ()

In [70]:
:info skipMany

class Alternative m => Parsing (m :: * -> *) where
  ...
  skipMany :: m a -> m ()
  ...
  	-- Defined in ‘Text.Parser.Combinators’

In [71]:
:t oneOf

oneOf :: forall (m :: * -> *). CharParsing m => [Char] -> m Char